# <center>CONNECT 4</center>

### README:
- the implementation uses Alpha-Beta pruning optimization technique for MinMax algorithm. It was tested and it gives good result
- dig_up method from SearchTreeNode class is used in the Runner after pruning search tree. It is used to search deeper. The idea is to create search tree to certain level, next to prune the search tree from the week branches, next dig up searching and again prune the result. This method improves preformance. Unfortunatelly, it does not gives alwasy the expected output. The expected output is first move at the middle of the board, as it guarantees the winning of the game. Max_depth for alpha-beta pruning is set up to 3, max_depth for dig_up is set up to 2, as this gives the expected output and has good performance.
- To play the game execute the Runner at the end of notebook. The method run() executes code in loop until the game is finished.


There are few performance tests. For that reason some of the code executes slowly.

In [1]:
import numpy as np
import copy
import sys

# START

In [2]:
# @param setup_of_board: 
# @param count_move: keep track of the number of made moves. When the game starts move=0
# @param index: index at the game board, into which the move was done. When the game starts there is no move, what is indicated by index=-1.
# @param player_at_this_level: keep trak of the player.
# @param win_pos_1: total number of possible winning positions for Player_1
# @param win_pos_2: total number of possible winning positions for Player_1
class CurrentBoard:
    # default CurrentBoard may be used in situation, when computer makes first move
    def __init__(self, setup_of_board=np.zeros(42, dtype=int), count_move=0, index=-1, player_at_this_level=2, win_pos_1=69, win_pos_2=69):
        # Check if board has the correct length
        if len(setup_of_board) != 42:
            print('Board has to have lenght (42)')
        else:
            self.board = setup_of_board # the current board. It also contains the current move made by player_at_this_level denoted by index
        self.board_width = 7
        self.board_height = 6
        self.board_total = self.board_width * self.board_height
        self.count_move = count_move # use count_move to check the draw situation
        self.index = index # the board index where the player_at_this_level made his move
        self.player_at_this_level = player_at_this_level
        self.winning_positions_1 = win_pos_1
        self.winning_positions_2 = win_pos_2
        
        # TODO : move to static variables
        self.player_1 = 1 # a value to be inserted to self.board denoting player_1 move
        self.player_2 = 2 # a value to be inserted to self.board denoting player_2 move
        self.win = False # True if it is a win situation
        self.draw = 0 # The value for draw situation. Draw is only after 42 move and there was not a winning situation
        
        # test index;
        # set up outcome;
        # check also if the current setup of the board is a winning situation -> done by game_won method
        if index >= 0 and index < self.board_total:
            # outcome after game_won function.
            # Value may be the value from util function (win situation) or eval function (depth)
            self.outcome = self.game_won(self.index) # Check utile/eval after every move

        # Situation when the board is clean (no moves)
        elif index == -1:
            self.outcome = 0
        else:
            print('Error - Index out of bound')
            
    # Returns number that represents the result of a current board
    # - sys.maxsize when it is a win situation
    # - sum of othe eval functions (position, trap)
    # @param - index where the next move will be done to the board
    # TODO: change name of the method
    # TODO: return the arrays of winning situation so all of them may be displayed (somethimes final move makes few winning situations)
    def game_won(self, index):
        # TODO: maybe remove it -> avoid situation when board[index]==0. We control index so there should be no such situation.
        if self.board[index] == 0:
            return 0
        
        # Check the win situation
        if self.game_won_at(index):
            self.win = True
            return sys.maxsize

        # Check the draw situation: use count_move to check the draw situation
        # The win situation is checked above so it is safe to check draw situation here
        if self.count_move == 42:
            return self.draw # The value is needed at the last move!!!
            
        ## eval functions
        eval_1 = self.eval_position()
        # Trap
        eval_2 = self.score_eq3_horizontal(index)
        eval_3 = self.eval_eq5_horiz_diag_right(index)
        eval_4 = self.eval_eq5_horiz_diag_left(index)
        
        return eval_1 + eval_2 + eval_3 + eval_4
        
    # Returns True if there is a win situation after inserting value at index to the board; return False otherwise
    #
    # @param index - the board index where the latest move was inserted
    def game_won_at(self, index):
        inc = 1 # Horizontal; increment by 1
        if ( self.count_from(index, inc) + self.count_from(index, -inc) ) > 4: # index is counted twice, that's why it is greater than 4
            return True

        inc = self.board_width # Vertical;
        if ( self.count_from(index, inc) + self.count_from(index, -inc) ) > 4:
            return True

        inc = self.board_width - 1 # Diagonal 1;
        if ( self.count_from(index, inc) + self.count_from(index, -inc) ) > 4:
            return True

        inc = self.board_width + 1 # Diagonal 2
        if ( self.count_from(index, inc) + self.count_from(index, -inc) ) > 4:
            return True

        return False

    # Returns the total number of 'neighbour' elements that are equal to the board element at start_index 
    # starting from start_index argument and incrementing by increment argument (negative or positive).
    # start_index is between 0 and board_total. It is valdate by constructor
    # If the method runs recursively validate it in other function -> done in equal_at method (next_move)
    #
    # @param start_index - the board index where the latest move was inserted
    # @param increment - the amount of positive or negative change in the start_index value
    def count_from(self, start_index, increment):
        # Test for edge. Apply only for horizontal and diagoanl situation. Exclude vertical situation (incrementation by board width)
        if (abs(increment) != self.board_width) and self.edge(start_index, start_index + increment): # modulo of vertical elements is 0
            return 1
        elif self.equal_at(start_index, start_index + increment):
            return 1 + self.count_from((start_index + increment), increment)           
        else:
            return 1

    # Returns True if two board item are equal or False otherwise
    #
    # @param start_index - the board index of element being compared to
    # @param next_index - the board index of other the element
    def equal_at(self, start_index, next_index):
        # validate if next_index is between 0 and board_total. start_index is already validated
        return ( next_index >= 0 ) and ( next_index < self.board_total ) and ( self.board[start_index] == self.board[next_index] )

    # Returns True if the element 'hit the edge' or False otherwise
    # This function may be allpied fo horizontal, diagonal 1 and diagonal 2 situation
    # Do not use this function to test vertical situation (subtraction of modulo of two vertical elements is 0)
    # 
    # @param i - the board index being compared to
    # @param j - the board other index     
    def edge(self, i, j):
        if abs(i%7 - j%7)!= 1: # modulo of horizontal, diagonal 1 and diagona 2 is 1. Modulo of horizontal is 0
            return True
        else:
            return False


# EVAL FUNCTIONS

    # Returns the difference between the number of possible winning positions between players
    def eval_position(self):
        self.update_winning_positions()
        
        if self.player_at_this_level == self.player_1:
            return self.winning_positions_1 - self.winning_positions_2
        else:
            return self.winning_positions_2 - self.winning_positions_1
        
        
    # PRIVATE FUNCTIONS FOR EVAL_POSITION METHOD
    
    def update_winning_positions(self):
        # update values of winning_positions_1 and winning_positions_2 
        # based on the player_at_this_level and the index (current_move)
        if self.player_at_this_level == self.player_1:
            self.winning_positions_2 = self.winning_positions_2 - self.total_blocked_positions(self.index)

        if self.player_at_this_level == self.player_2:
            self.winning_positions_1 = self.winning_positions_1 - self.total_blocked_positions(self.index)
    
    # Returns the number of blocked winning positions after inserting value at index to the board
    # It does not count already blocked winning positions made by earlier move.
    #
    # @param index - the board index where the latest move was inserted
    def total_blocked_positions(self, index):
        total_blocked_positions = 0
        start_value = self.board[index] # TODO could be replaced with player_at_this_leve
        # start_value = player_at_this_level # TODO test at first
        
        inc = 1 # Horizontal; increment by 1
        total_blocked_positions = total_blocked_positions + self.count_blocked_positions(index, inc, start_value)
        
        inc = self.board_width # Vertical;
        total_blocked_positions = total_blocked_positions + self.count_blocked_positions(index, inc, start_value)

        inc = self.board_width - 1 # Diagonal 1;
        total_blocked_positions = total_blocked_positions + self.count_blocked_positions(index, inc, start_value)

        inc = self.board_width + 1 # Diagonal 2
        total_blocked_positions = total_blocked_positions + self.count_blocked_positions(index, inc, start_value)

        return total_blocked_positions
    
    # Returns the number of blocked positions horizontaly, verticaly or diagonally
    # @param index - 
    # @param inc - 
    # @param #  TODO choose one of them: start_value/player_at_this_level  - 
    def count_blocked_positions(self, index, inc, start_value):
        blocked_positions = 0
        
        # spread - the number of indices the winning positions may be build from
        spread_inc = self.eval_count_from(index, inc, start_value)
        spread_decr = self.eval_count_from(index, -inc, start_value)
        # spread_inc and spread_decr can not be bigger that 4
        if spread_inc > 4:
            spread_inc = 4
        if spread_decr > 4:
            spread_decr = 4
        
        spread = spread_inc + spread_decr - 1 # spread of index is counted twice, that's why subtract by 1
        if spread >= 4:
            blocked_positions = blocked_positions + spread%4 + 1
        
        return blocked_positions
    
    # HELPER FUNCTIONS FOR count_blocked_positions METHOD

    # Returns the total number of 'neighbour' elements that are equal to the board element at start_index 
    # starting from start_index argument and incrementing by increment argument (negative or positive).
    # start_index is between 0 and board_total. It is also validated for range in equal_at
    # 
    # Do not count more than 4 elements
    # 
    # @param start_index - the board index where the latest move was inserted
    # @param increment - the amount of positive or negative change in the start_index value
    # TODO: change name of the method
    def eval_count_from(self, start_index, increment, start_value):
        # validate next index - it must be validated here as we increment or decrement index
        # start_index was already validated before making input to the board
        # but if next_index is not valid, return 1 (count the start_index)
        if not self.valid_index(start_index + increment):
            return 1
        
        # Test for edge. Apply only for horizontal and diagoanl situation. Exclude vertical situation (incrementation by board width)
        if (abs(increment) != self.board_width) and self.edge(start_index, start_index + increment): # modulo of vertical elements is 0
            return 1
        elif not self.eval_equal_at(start_value, start_index + increment):
            return 1 + self.eval_count_from(start_index + increment, increment, start_value)
        else:
            return 1

    # Returns True if two board item are equal or False otherwise
    #
    # @param start_index_value - the value to compare to
    # @param next_index - the board index of other the element
    # TODO: change name of the method
    def eval_equal_at(self, start_index_value, next_index):
        return start_index_value == self.board[next_index]

    # Returns True if index is between 0 and 42
    def valid_index(self, index):
        return ( index >= 0 ) and ( index < self.board_total )


    # EVAL FUNCTION - SEARCH FOR A TRAP
    
    # Returns score value for the situation, that the move creates winning situation: 3 in a row and to available slots from both sides.
    # There is 6 possible situations.
    # TODO: what about situation when the columns of empty slots are below the level of 3???
    # TODO: refactor the code; use similar way to eval_position method
    def score_eq3_horizontal(self, index):
        score = 0
        if (index%7 == 1) or (index%7 == 2) or (index%7 == 3):
            if ((self.board[index - 1]) == 0) and (self.board[index+1] == self.player_at_this_level) and (self.board[index+2] == self.player_at_this_level) and (self.board[index+3] == 0):
                score = 100000
        elif (index%7 == 3) or (index%7 == 4) or (index%7 == 5):
            if (self.board[index + 1] == 0) and (self.board[index-1] == self.player_at_this_level) and (self.board[index-2] == self.player_at_this_level) and (self.board[index-3] == 0):
                score = 100000
        return score

    # Returns score value for the situation, that the move creates winning situation: 3 in a row and diagonal right up 
    # and slot available 0n right.
    # There is 3 main possible situations for inserting
    # TODO: Test it
    def eval_eq5_horiz_diag_right(self, index):
        score = 0
        v = self.player_at_this_level
        empty = 0
        
        if index  < 14: # avoid index out of range
            return score
        
        # situation 1 - insert on the left
        if (index%7 < 4):
            if (self.board[index+3] == empty) and (self.board[index+1] == v) and (self.board[index+2] == v) and (self.board[index-14] == v) and (self.board[index-6] == v):
                score = 1000000
            
        # situation 2 - insert in the middle
        if (index%7 < 5) and (index%7 > 0):
            if (self.board[index+2] == empty) and (self.board[index-1] == v) and (self.board[index+1] == v) and (self.board[index-15] == v) and (self.board[index-7] == v):
                score = 1000000
            
        # situation 3 - insert on the right
        if (index%7 < 6) and (index%7 > 1):
            if (self.board[index+1] == empty) and (self.board[index-2] == v) and (self.board[index-1] == v) and (self.board[index-16] == v) and (self.board[index-8] == v):
                score = 1000000

        return score
    
    
    # Returns score value for the situation, that the move creates winning situation: 3 in a row and diagonal left up 
    # and slot available 0n right.
    # There is 3 main possible situations for inserting
    # TODO: Test it
    def eval_eq5_horiz_diag_left(self, index):
        score = 0
        v = self.player_at_this_level
        empty = 0
        
        if index  < 15: # avoid index out of range
            return score
        
        # situation 1 - insert on the right
        if (index%7 > 2):
            if (self.board[index-3] == empty) and (self.board[index-1] == v) and (self.board[index-2] == v) and (self.board[index-14] == v) and (self.board[index-8] == v):
                score = 1000000
            
        # situation 2 - insert in the middle
        if (index%7 > 1) and (index%7 < 6):
            if (self.board[index-2] == empty) and (self.board[index-1] == v) and (self.board[index+1] == v) and (self.board[index-13] == v) and (self.board[index-7] == v):
                score = 1000000
            
        # situation 3 - insert on the left
        if (index%7 < 0):
            if (self.board[index-1] == empty) and (self.board[index+1] == v) and (self.board[index+2] == v) and (self.board[index-12] == v) and (self.board[index-6] == v):
                score = 1000000

        return score
    
        
    # Returns True if the board is symetric or False otherwise
    # The symetric move may occure only on symetric board. Create only left side or right side moves on symetric board
    # TODO: right or left - could be random
    # TODO: test this function
    def symetric_board(self):
        for i in range(6):
            if self.board[0+i*7] != self.board[6+i*7]:
                return False
            if self.board[1+i*7] != self.board[5+i*7]:
                return False
            if self.board[2+i*7] != self.board[4+i*7]:
                return False
        return True

    # Returns the list of CurrentBoard or None containing all possible moves at this stage of the game.
    # The list have size of 7 (7 columns).
    # None value in the list indicates, that all moves were done for a given column.
    # Loop through rows and check if the slot is available. It is availabel if the value is greater than 0. 
    def all_possible_moves_for(self):
        list_of_all_possible_moves = np.empty(7, CurrentBoard) # List of CurrentBoard objects, initialized with None values
        
        for col in range(7):
            for row in range(6):
                index = col + 7 * row
                if self.board[index] == 0:
                    temp_board = copy.deepcopy(self.board)
                    temp_board[index] = self.other() # Insert the move done by player
                    new_current_board = CurrentBoard(temp_board, self.count_move + 1, index=index, player_at_this_level=self.other())
                    list_of_all_possible_moves[col]= new_current_board # append CurrentBoard object
                    break
        return list_of_all_possible_moves
        
    # Displays Game Board filled up with made moves and a row on top of a board indicating the column number.
    # The player choose the column number to make a move
    def display_game_board(self):
        temp_board = np.flipud(self.board.reshape(6, 7)).reshape(42) # Reshapes, flips up/down, and reshapes board
        
        print('| 1| 2| 3| 4| 5| 6| 7|')
        print('======================')
        for index, value in enumerate(temp_board):
            # Change the value of variable to 0 if it is greater than 0
            if value == 0:
                value = ''
                
            print('|', end = '')
            if((index + 1)%7 == 0):
                print('{:2}'.format(value), end = '|')
                print('\n----------------------')
            else:
                print('{:2}'.format(value), end = '')
    
    # Displays board with values
    # values: 1 playre_1, 2 player_2, 0 available spaces
    def display(self):
        temp_board = np.flipud(self.board.reshape(6, 7)).reshape(42) # Reshapes, flips up/down, and reshapes board
        
        for index, value in enumerate(temp_board):
            print('|', end = '')
            if((index + 1)%7 == 0):
                print('{:2}'.format(value), end = '|')
                print('\n----------------------')
            else:
                print('{:2}'.format(value), end = '')
        
    # Returns 'other' player character
    def other(self):
        if self.player_at_this_level == self.player_1:
            return self.player_2
        if self.player_at_this_level == self.player_2:
            return self.player_1


In [3]:
class SearchTreeNode:
    def __init__(self, Current_Game = None, ply_depth = 0, max_depth = 4, comp_first=True):
        self.children = [] # A list of SearchTreeNode. It it a list of all possible moves until given ply_depth
        self.ply_depth = ply_depth
        self.max_depth = max_depth
        self.node_terminal = False # variable used in generate_children() and min_max() function
        # TODO: node_trap - generate children until node_terminal when there is a node_trap
        self.node_trap = False # variable that indicates if this node has a move_trap or not
        self.node_value = None # assigned by MinMax
        if Current_Game == None:
            # Type CurrentBoard
            self.game_at_this_node = CurrentBoard() # Starting point, player_1 starts as first
        else:
            self.game_at_this_node = Current_Game
            
        # Control who starts first. This variable is used to determine if it is a winning ar loosing situation
        # comp_first works in tandem with CurrentBoard.count_move
        self.comp_first = comp_first
            
        # Call generat_children
        self.generate_children() # if win, loose or draw situation, assign True to the node_terminal
        
    # Generates SearchTreeNode of a given max_depth
    # Generates all possible nodes for the game_at_this_node (type CurrentBoard)
    # Terminal states: stop generating child node:
    # - when there is win/lose situation - the value of game_at_this_node.win == True.
    # - when there is draw situation - the value of game_at_this_node.count_move == 42 and there is no win/lose situation.
    # - if the ply_depth is max_depth.
    # Assigns node_value on terminal state:
    # - node_value is equal to game_at_this_node.outcome if it is comp move
    # - node value is equal to minus game_at_this_node.outcome if it is user move
    # TODO - Restart ply_depth after choosing the next move, it means choosing the next CurrentBoard, in a game class
    # REMEMBER: no need to check if it is max or min level when generating children, as an negativ outcome is return in min_max function.
    def generate_children(self):
        # the base case is when:
        # - all_possble_moves_for() returns an empty list
        # - or there is a winning situation
        # - or ply_depth is greater than max_depth
        # Winning move - you can not make a move and loose, you can either win or draw
        ### TODO Do not miss out winning score -> will not, because at first the CurrentBoard.outcome checks win, lose, draw, next the eval
        # checks win/lose/draw situation
        if self.game_at_this_node.win or (self.game_at_this_node.count_move == 42):
            self.node_terminal = True
# TEST
#                print('STN.generate_children(): ply_depth {}; isComp {}; node_value {} '.format(self.ply_depth, self.isCompMove(), self.node_value))
#            else:
#                self.node_value = -self.game_at_this_node.outcome
# TEST
#                print('STN.generate_children(): ply_depth {}; isComp {}; node_value {} '.format(self.ply_depth, self.isCompMove(), self.node_value))
#            self.is_node_value_assigned = True
            
        # create new nodes
        elif self.ply_depth < self.max_depth:
            # get all possible moves at this stage
            list_of_all_possible_moves = self.game_at_this_node.all_possible_moves_for() # size=7

            # Check if board is symetric. 
            # If it is, assign None value to the left or right side of the list_of_all_possible_moves.
            # Keep the size=7 -> assign None to the values that are symetric.
            # TODO or take random from left or right side
            if self.game_at_this_node.symetric_board():
                list_of_all_possible_moves[4:] = None
            
            # loop through all_moves (type CurrentBoard), create child node as a SearchTreeNode type and appent it to the children list
            for new_game in list_of_all_possible_moves:
                if new_game != None:               
                    self.children.append(SearchTreeNode(Current_Game=new_game, ply_depth=self.ply_depth + 1, max_depth=self.max_depth, comp_first=self.comp_first))
    
            # Optimization for min_max_alpha_beta algorithm. Enable better pruning. Move child that has better outcome to the left of a SearchTreeNode.
            self.children = sorted(self.children, key=lambda x: x.game_at_this_node.outcome, reverse=True)

            # TODO: From sorted children choose the first 4 to the next SearchNodeTree. Avoid unnecessary creation of not promising nodes
    
    
    # Assigns a value to the node_value variable.
    # Loop recursively over search tree node. BaseCase: leaf
    def min_max(self, is_max=True):
        # Check if it is a leaf. A length of children list of leaf is 0.
        if self.isLeaf(): # BaseCase: if it is a leaf return the game outcome
            if self.isCompMove():
                self.node_value = self.game_at_this_node.outcome
            else:
                self.node_value = -self.game_at_this_node.outcome
        else: # assign a max or min node_value from the children to the parent node_value
            # Check if it is max or min layer
            if is_max: # max - return the max node_value of children
                self.node_value = -sys.maxsize
                for child in self.children:
                    # Assign values to the childrens' node_value
                    child.min_max(is_max=False) # Call min_max() function recursively
                    # get child.node_value and check the condition
                    if child.node_value > self.node_value:
                        self.node_value = child.node_value
            else: # min - return the min node_value of children
                self.node_value = sys.maxsize
                for child in self.children:
                    # Assign values to the childrens' node_value
                    child.min_max(is_max=True) # Call min_max() function recursively                
                    # get child.node_value and check the condition
                    if child.node_value < self.node_value:
                        self.node_value = child.node_value
            
    # Assigns a value to the node_value variable.
    # Loop recursively over search tree node. BaseCase: leaf
    def min_max_alpha_beta(self, is_max=True, alpha=-sys.maxsize, beta=sys.maxsize):
        # Check if it is a leaf. A length of children list of leaf is 0.
        if self.isLeaf(): # BaseCase: if it is a leaf return the game outcome
            if self.isCompMove():
                self.node_value = self.game_at_this_node.outcome
            else:
                self.node_value = -self.game_at_this_node.outcome
        else:
            # Check if it is max or min layer
            if is_max: # max - return the max node_value of children
                self.node_value = -sys.maxsize
                for index, child in enumerate(self.children):
                    # Assign values to the childrens' node_value
                    child.min_max_alpha_beta(is_max=False, alpha=alpha, beta=beta) # Call min_max() function recursively
                    # get child.node_value and check the condition
                    if child.node_value > self.node_value:
                        self.node_value = child.node_value
                        # alpha beta pruning
                        alpha = self.node_value
                        if beta <= alpha:
                            # Remove the rest of children from the list
                            removed = self.children[index+1:]
                            self.children = self.children[:index+1]
#                            print('Prunned: alpha {}; beta {}; removed child {}'.format(alpha, beta, removed))
                            break
            else: # min - return the min node_value of children
                self.node_value = sys.maxsize
                for index, child in enumerate(self.children):
                    # Assign values to the childrens' node_value
                    child.min_max_alpha_beta(is_max=True, alpha=alpha, beta=beta) # Call min_max() function recursively                
                    # get child.node_value and check the condition
                    if child.node_value < self.node_value:
                        self.node_value = child.node_value
                        beta = self.node_value
                        if beta <= alpha:
                            # Remove the rest of children from the list
                            removed = self.children[index+1:]
                            self.children = self.children[:index+1]
#                            print('Prunned: alpha {}; beta {}; removed child {}'.format(alpha, beta, removed))
                            break

        

    # Returns the nodes (SearchTreeNode) from children list which outcome is equal to the value of node_value
    # The outcome for children of min layer the value of node_value = -child.outcome
    def get_all_next_moves(self):
        next_moves = []
        for child in self.children:
            if child.node_value == self.node_value:
                next_moves.append(child)
        return next_moves
    
    # Reurns the best node (SearchTreeNode) from children as a next move
    # TODO correct this function
    def get_next_move(self):
        next_moves = self.get_all_next_moves()
        # TODO: return the random next move
        return next_moves[0]
        
    
    # Deepens the search tree
    # depth can not be greater than max_depth
    def dig_up(self, depth=None):
        if depth == None:
            depth = self.max_depth
        # go to the leaf and generate children
        # Do not generate children for win situation
        # Do not generate children for trap. Generate children for trap once the game is in trap node.
        # Trap node is is controled by self.node_trap
        for child in self.children:
            if not child.isLeaf() and not child.game_at_this_node.win:
                child.dig_up(depth)
            elif child.isLeaf() and not child.game_at_this_node.win:
                child.ply_depth = child.max_depth - depth
                child.generate_children()

    # Return True if the node is a leaf (do not have children)
    def isLeaf(self):
        return len(self.children) == 0

    
# TODO: refactor the below methods. They are the same

    # Return True if maximizer player, False if minimizer player.
    def isMax(self):        
        # count_move is taken from the CurrentBoard of a node
        count_move = self.game_at_this_node.count_move
        
        if self.comp_first:
            return (count_move%2 == 0)
        else:
            return (count_move%2 == 1)

    
    # Helper function for generate_children(), min_max() and min_max_alpha_beta()
    # Return True if it is a comp move at this level, False if it is user move.
    # If comp starts as first, the comp move is on odd count_move
    # If comp start as second, the comp move is on even count_move
    def isCompMove(self): # computer
        # count_move is taken from the CurrentBoard of a node
        count_move = self.game_at_this_node.count_move
        
        if self.comp_first:
            return (count_move%2 == 1)
        else:
            return (count_move%2 == 0)
        
    # Helper function for min_max() and min_max_alpha_beta()
    # Return True if maximizer player, False if minimizer player.
    # The maximizer tries to get the highest score possible while the minimizer tries to do the opposite and get the lowest score possible.
    def isMaximizer(self):
        return not self.isCompMove()


            
    # Returns random SearchTreeNode from children if there are more than one child with equal node_value
    # TODO test this method
    def select_child(self):
        # list of indices from children[] for which the child have equal node_value
        indices = []
        # Loop over the list of children to select all child indices having equal node_value
        for index, child in enumerate(self.children):
            if child.node_value == self.next_move.node_value:
                indices.append(index)
        # select random index from the list of child with equal node_value
        # return child with the given index
        return self.children[choice(indices)]

    # Returns the list of size 7 containing all possible moves for a CurrentBoard.
    # Returns indices of all possible moves at this level.
    # If there is no more availabel moves for a given column, the corresponding value for that column in the list is None
    # TODO test it
    def all_possible_indices(self):
        # Initialize list of None of size 7
        temp_array = [None for i in range(7)] # List Comprehensions
        board = self.game_at_this_node.board
        for i, v in enumerate(temp_array):
            # TODO make it as a loop / if condition
            for row in range(6):
                if board[i+7*row] == 0:
                    temp_array[i] = i+7*row
                    break
        return temp_array

    # The helper method to test if alpa-beta pruning works
    # Returns the size of Tree - 1
    # TODO add 1 to the root node (add root node)
    def tree_size(self):
        size = 0
        if not self.isLeaf():
            for child in self.children:
                size += child.tree_size()
        
        size += len(self.children)
        return size
        


In [4]:
# Game Runner
# 'X' has first move
class Runner:
    def __init__(self, max_depth=3):
        self.user_name = 'Adam'
        self.player_1 = 1 # starts first
        self.player_2 = 2
        self.comp_first = True # boolean
        self.game_search_tree_node = None # type SearchTreeNode
        self.current_board = None # type CurrentBoard
        self.result = None # possible results: 'w' - win, 'l' - loss, 'd' - draw
        
        self.user_char = None
        self.comp_char = None
        
        self.max_depth = max_depth
        
    def run(self):
        self.set_up_game()
        self.first_move()
        self.loop_game()
        self.end_game()
    
    # Set up user_name, user_char, comp_first (user name, character user plays, if the comp has first move)
    def set_up_game(self):
        self.user_name = input('Hello, insert your name')
        self.comp_first = (input('Would you like to start Y/N').upper() != 'Y')
        
        if self.comp_first:
            self.comp_char = self.player_1
            self.user_char = self.player_2
        else:
            self.comp_char = self.player_2
            self.user_char = self.player_1
    
    # Set up initial current_board, game_search_tree_node depending on user_first (who has first move - user or computer)
    def first_move(self):
        # Avoid running unnecessary SearchTreeNode (very computing expensive). If user moves as first, take user input and create CurrentBoard with this move. Next create SearchTreeNode
        
        # Set up CurrentBoard for first computer move, but first or second move in the game
        if self.comp_first: # Computer starts first
            self.current_board = CurrentBoard() # use default CurrentBoard
        else: # User starts first
            # Display empty board
            self.current_board = CurrentBoard(setup_of_board=np.zeros(42, dtype=int), count_move=0, index=-1, player_at_this_level=self.player_2)
            self.current_board.display_game_board()
            # initialize game_search_tree_node with max_depth=0. The game_search_tree_node is needed for validate_input()
            self.game_search_tree_node = SearchTreeNode(Current_Game=self.current_board, ply_depth = 0, max_depth=0, comp_first=self.comp_first)
            
            print('{}, Your move. Choose column by inserting the number from 1 to 7. Do not use the column if it is full>'.format(self.user_name))
            user_choice = self.validate_input() # get user input
            next_board = self.current_board.board # cache board (numpy array)
            next_board[user_choice] = self.user_char # update board
            # Create CurrentBoard from next_board and cache it
            self.current_board = CurrentBoard(setup_of_board=next_board, count_move=(self.current_board.count_move + 1), index=user_choice, player_at_this_level=self.user_char)
            
            # Display current board for user
            print('Board after User move is:')
            self.current_board.display_game_board()
            # TEST
            print('In a first_move method. User Move (user first) - count_move: {}; outcome is: {}; comp_char: {}, comp_first {}, user_char: {} '.format(self.current_board.count_move, self.current_board.outcome, self.comp_char, self.comp_first, self.user_char))

            # TODO to make the game more intresting if user start first with the optimal move, chose the random comp move
            # next User choses his move, next generate game_search_tree_node
            
            
        # Create SearchTreeNode and assign it to self.game_search_tree_node
        # TODO: bug with the dig_up(): it doesn't give the optimal first move 
        self.game_search_tree_node = SearchTreeNode(Current_Game=self.current_board, ply_depth = 0, max_depth = self.max_depth, comp_first=self.comp_first)
        # Call min_max function
        
# Standard MinMax function
#        self.game_search_tree_node.min_max(is_max=self.game_search_tree_node.isMax())

        self.game_search_tree_node.min_max_alpha_beta(is_max=self.game_search_tree_node.isMax())
        # Call dig_up()
        self.game_search_tree_node.dig_up(depth=2) # 2 is fast and gives optimal first move
        # Call min_max function
        self.game_search_tree_node.min_max_alpha_beta(is_max=self.game_search_tree_node.isMax())
    

    # Run game in a loop until it is resolved.
    # Follow the sequnce: comp move -> checking result -> displaying board after computer move to the user -> 
    # -> user move -> checking result ->  displaying board after user move
    def loop_game(self):
        run = True
        while run:
            # COMPUTER MOVE
            self.game_search_tree_node = self.game_search_tree_node.get_next_move()
            
            # Cache current_board -> the best computer move
            self.current_board = self.game_search_tree_node.game_at_this_node

            # TODO: remove it -> prints for testing purpose
            print('In a loop_game method. Computer Move - count_move: {}; outcome is: {}; comp_char: {}, '.format(self.current_board.count_move, self.current_board.outcome, self.comp_char))

            # Check draw
            if self.current_board.count_move == 42 and not self.current_board.win:
                self.result = 'draw'
                break;

            # Check win situation
            if self.current_board.win:
                run = False
                self.result = 'comp won'
                break;

            # Display board after computer move
            print('Board after computer move is:')
            self.current_board.display_game_board()
            
            # USER MOVE
            # Create next_board based on board of the current_board
            print('{}, it is your turn. Choose column >> '.format(self.user_name))
            user_choice = self.validate_input() # get user input
            next_board = self.current_board.board # cache board (numpy array)
            next_board[user_choice] = self.user_char # update board
            
            # Search children that has equal board as the choosen by the User and reassign SearchTreeNode with the current one in game_search_tree_node
#            node_exist = False # variable to store if child node exist so the parent node may be replaced
#            for child in self.game_search_tree_node.children:
#                if np.array_equal(child.game_at_this_node.board, next_board):
#                    self.game_search_tree_node = child # Replace game_search_tree_node with existing child node
#                    node_exist = True
#                    break
            
            # TODO test eficiency nad correctness. Is it enough to refresh search_tree_node every second loop? 
            # Create new search_tree_node if the chosen no by the user doesn't exist or every second loop (refresh the search_tree_node)
#            if (not node_exist) or (counter > 1):
                # Create CurrentBoard from next_board
#                next_game = CurrentBoard(setup_of_board=next_board, count_move=(self.current_board.count_move + 1), index=user_move, player_at_this_level=self.user_char)
                # Create SearchTreeNode and assign it to self.game_search_tree_node
#                self.game_search_tree_node = SearchTreeNode(Current_Game=next_game, ply_depth = 0, max_depth=self.max_depth, comp_first=self.comp_first)
                # Call min_max function
#                self.game_search_tree_node.min_max_alpha_beta(is_max=self.game_search_tree_node.isMax())
                # Call dig_up()
#                self.game_search_tree_node.dig_up(depth=self.max_depth-1)
                # Call min_max function
#                self.game_search_tree_node.min_max_alpha_beta(is_max=self.game_search_tree_node.isMax())
#                counter = 0 # reset counter

            # Create CurrentBoard from next_board and cache it
            self.current_board = CurrentBoard(setup_of_board=next_board, count_move=(self.current_board.count_move + 1), index=user_choice, player_at_this_level=self.user_char)

            # Optimization: change the value of self.max_depth. Search deeper every 14 move
            if self.current_board.count_move%14 == 0:
                self.max_depth += 1
            
            # Create SearchTreeNode and assign it to self.game_search_tree_node
            self.game_search_tree_node = SearchTreeNode(Current_Game=self.current_board, ply_depth = 0, max_depth=self.max_depth, comp_first=self.comp_first)
            # Call min_max function
            self.game_search_tree_node.min_max_alpha_beta(is_max=self.game_search_tree_node.isMax())
            # Call dig_up()
            self.game_search_tree_node.dig_up(depth=self.max_depth) # TODO test even and odd deep
            # Call min_max function
            self.game_search_tree_node.min_max_alpha_beta(is_max=self.game_search_tree_node.isMax())
                                
            # check draw
            if self.current_board.count_move == 42 and not self.current_board.win:
                self.result = 'draw'
                break;

            # Check win situation
            if self.current_board.win:
                run = False
                self.result = 'user won'
                break;

            # TODO: remove it -> prints for testing purpose
            print('User Move - count_move: {}; outcome is: {}; comp_char: {}, player_at_this_level {}'.format(self.current_board.count_move, self.current_board.outcome, self.comp_char, self.current_board.player_at_this_level))
                
            # Display current board for user
            print('Board after User move is:')
            self.current_board.display_game_board()

    # TODO: correct code
    # Display final result
    def end_game(self):
        print("Game over")
        self.game_search_tree_node.game_at_this_node.display_game_board()
        if self.result == 'user won':
            print(self.user_name, ', congratulations, you have won')
        elif self.result == 'comp won':
            print(self.user_name, ', you have lost')
        elif self.result == 'draw':
            print(self.user_name, ', it is draw')
        else:
            print(self.user_name, ', it is a BUG!!!!')
            

    # Returns a valid index from the CurrentBoard.board array after User input validation.
    # User can choice the number from 1 to 7 inclusive. Numbers from 1 to 7 represent the columns' numbers.
    # Hence, the valid number is from 1 to 7, excluding the full columns.
    def validate_input(self):
        user_choice = None
        # get the list of all possible moves, size=7. The list contains None if the column is full or the board index
        all_possible_indices = self.game_search_tree_node.all_possible_indices()
        # run loop until the input is valid
        run = True
        while(run):
            try:
                user_choice = int(input()) - 1 # shift left by 1, as arrays starts from 0
                if (user_choice < 0) or (user_choice > 6):
                    print('The input must be between 1 and 7 >>')
                elif all_possible_indices[user_choice] == None:
                    print('No more moves for the choosen column. Choose other one >>')
                else:
                    run = False
            except ValueError:
                print("The input is not a number. It's a string")
        return all_possible_indices[user_choice]


# TESTS

# TEST CURRENTBOARD CLASS

## Tests:
- create CurrentBoard object
- display()
- display_game_board()

### Test 1) default CurrentBoard

In [5]:
# Expected: 
#'board': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
# 'count_move': 0,
# 'index': -1,
# 'player_at_this_level': 2,
# 'player_1': 1,
# 'player_2': 2,
# 'win': False,
# 'outcome': 0

cb = CurrentBoard()
vars(cb)

{'board': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'board_width': 7,
 'board_height': 6,
 'board_total': 42,
 'count_move': 0,
 'index': -1,
 'player_at_this_level': 2,
 'winning_positions_1': 69,
 'winning_positions_2': 69,
 'player_1': 1,
 'player_2': 2,
 'win': False,
 'draw': 0,
 'outcome': 0}

In [6]:
cb.display()

| 0| 0| 0| 0| 0| 0| 0|
----------------------
| 0| 0| 0| 0| 0| 0| 0|
----------------------
| 0| 0| 0| 0| 0| 0| 0|
----------------------
| 0| 0| 0| 0| 0| 0| 0|
----------------------
| 0| 0| 0| 0| 0| 0| 0|
----------------------
| 0| 0| 0| 0| 0| 0| 0|
----------------------


In [7]:
cb.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------


### Test 2) board of containing sample 1 na 2 values

In [8]:
board=np.array([1, 1, 2, 1, 1, 2, 1,
                1, 0, 1, 1, 1, 1, 0,
                1, 0, 0, 1, 1, 0, 0,
                1, 0, 0, 1, 1, 0, 0,
                0, 0, 0, 0, 0, 0, 0,
                0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(setup_of_board=board, count_move=18, index=21, player_at_this_level=1)
vars(cb)

{'board': array([1, 1, 2, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
        0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'board_width': 7,
 'board_height': 6,
 'board_total': 42,
 'count_move': 18,
 'index': 21,
 'player_at_this_level': 1,
 'winning_positions_1': 69,
 'winning_positions_2': 69,
 'player_1': 1,
 'player_2': 2,
 'win': True,
 'draw': 0,
 'outcome': 9223372036854775807}

In [9]:
cb.display()

| 0| 0| 0| 0| 0| 0| 0|
----------------------
| 0| 0| 0| 0| 0| 0| 0|
----------------------
| 1| 0| 0| 1| 1| 0| 0|
----------------------
| 1| 0| 0| 1| 1| 0| 0|
----------------------
| 1| 0| 1| 1| 1| 1| 0|
----------------------
| 1| 1| 2| 1| 1| 2| 1|
----------------------


In [10]:
cb.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
| 1|  |  | 1| 1|  |  |
----------------------
| 1|  |  | 1| 1|  |  |
----------------------
| 1|  | 1| 1| 1| 1|  |
----------------------
| 1| 1| 2| 1| 1| 2| 1|
----------------------


## Test all_possible_moves_for()

### Test 1) default CurrentBoard

In [11]:
# Expected: returns a list of all possible moves. Should be 7 object of CurrentBoard

cb = CurrentBoard()
all_moves = cb.all_possible_moves_for()
all_moves

array([<__main__.CurrentBoard object at 0x0000022AD9B44A58>,
       <__main__.CurrentBoard object at 0x0000022AD9BA75C0>], dtype=object)

In [12]:
# Expected: a board with move done by player_1 - value 1 in one of the field
cb.all_possible_moves_for()[0].display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
| 1|  |  |  |  |  |  |
----------------------


In [13]:
# Test index property in the list of all_moves.
# Expected: 0 1 2 3 4 5 6
for move in all_moves:
    if move != None:
        print(move.index, end=(' '))

0 1 2 3 4 5 6 

### Test 2) columns filled up on different level

In [14]:
board=np.array([1, 0, 2, 1, 1, 2, 1,
                1, 0, 1, 1, 1, 1, 0,
                1, 0, 0, 1, 1, 0, 0,
                2, 0, 0, 1, 1, 0, 0,
                1, 0, 0, 0, 0, 0, 0,
                2, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(setup_of_board=board, count_move=18, index=21, player_at_this_level=1)
cb.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
| 2|  |  |  |  |  |  |
----------------------
| 1|  |  |  |  |  |  |
----------------------
| 2|  |  | 1| 1|  |  |
----------------------
| 1|  |  | 1| 1|  |  |
----------------------
| 1|  | 1| 1| 1| 1|  |
----------------------
| 1|  | 2| 1| 1| 2| 1|
----------------------


In [15]:
# Expected: None, CurrentBoard, CurrentBoard, CurrentBoard, CurrentBoard, CurrentBoard, CurrentBoard
all_moves = cb.all_possible_moves_for()
all_moves

array([None, <__main__.CurrentBoard object at 0x0000022AD9BA7358>,
       <__main__.CurrentBoard object at 0x0000022AD9BA7668>], dtype=object)

In [16]:
# Expected: 1 16 31 32 19 13 

all_moves = cb.all_possible_moves_for()
for move in all_moves:
    if move != None:
        print(move.index, end=(' '))

1 16 31 32 19 13 

## Test equal_at(self, start_index, next_index)

In [17]:
# test parameters: for 0, 1, 2, 3, 4, 5, 6
# Expected: - 0, 2, 4, 6 result is False; 1, 3, 5 result is True

board = np.array([ 1, 1, 2, 2, 0, 0, 1,
                     1, 0, 1, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0,
                     0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
for i in [0, 1, 2, 3, 4, 5, 6]:
    print('Parameter: {} - value: {}'.format(i, cb.equal_at(i, i + 1)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
| 1|  | 1|  |  |  |  |
----------------------
| 1| 1| 2| 2|  |  | 1|
----------------------
Parameter: 0 - value: True
Parameter: 1 - value: False
Parameter: 2 - value: True
Parameter: 3 - value: False
Parameter: 4 - value: True
Parameter: 5 - value: False
Parameter: 6 - value: True


## Test count_from(self, start_index, increment)

### Test count_from() - horizontally

In [18]:
board = np.array([ 1, 1, 1, 1, 0, 2, 2,
                   2, 2, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected: 
# 0 result is 4
# 1 result is 3
# 2 result is 2
# 3 result is 1
# 5 result is 2 # test edge
print('Test with positive argument for increment')
for i in [0, 1, 2, 3, 5]:
    print('Parameter: {} - value: {}'.format(i, cb.count_from(i, increment=1)))
    
print('Test with negative argument for increment')
# Expected: 
# 0 result is 1
# 1 result is 2
# 2 result is 3
# 3 result is 4
# 8 result is 2 # test edge
for i in [0, 1, 2, 3, 8]:
    print('Parameter: {} - value: {}'.format(i, cb.count_from(i, increment=-1)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
| 2| 2|  |  |  |  |  |
----------------------
| 1| 1| 1| 1|  | 2| 2|
----------------------
Test with positive argument for increment
Parameter: 0 - value: 4
Parameter: 1 - value: 3
Parameter: 2 - value: 2
Parameter: 3 - value: 1
Parameter: 5 - value: 2
Test with negative argument for increment
Parameter: 0 - value: 1
Parameter: 1 - value: 2
Parameter: 2 - value: 3
Parameter: 3 - value: 4
Parameter: 8 - value: 2


### Test count_from() - verticaly

In [19]:
board = np.array([ 0, 0, 0, 0, 0, 0, 2,
                   0, 0, 0, 0, 0, 0, 2,
                   0, 0, 0, 0, 0, 0, 2,
                   0, 0, 0, 0, 0, 0, 2,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected: 
# 6 result is 4
# 13 result is 3
# 20 result is 2
# 27 result is 1
print('Test with positive argument for increment by 7')
for i in [6, 13, 20, 27]:
    print('Parameter: {} - value: {}'.format(i, cb.count_from(i, increment=7)))
    
print('Test with negative argument for increment by -7 - decrement')
# Expected: 
# 6 result is 1
# 13 result is 2
# 20 result is 3
# 27 result is 4
for i in [6, 13, 20, 27]:
    print('Parameter: {} - value: {}'.format(i, cb.count_from(i, increment=-7)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  | 2|
----------------------
|  |  |  |  |  |  | 2|
----------------------
|  |  |  |  |  |  | 2|
----------------------
|  |  |  |  |  |  | 2|
----------------------
Test with positive argument for increment by 7
Parameter: 6 - value: 4
Parameter: 13 - value: 3
Parameter: 20 - value: 2
Parameter: 27 - value: 1
Test with negative argument for increment by -7 - decrement
Parameter: 6 - value: 1
Parameter: 13 - value: 2
Parameter: 20 - value: 3
Parameter: 27 - value: 4


### Test count_from() - diagonal right up

In [20]:
board = np.array([ 0, 1, 0, 0, 0, 2, 0,
                   0, 0, 1, 0, 0, 0, 2,
                   0, 0, 0, 1, 0, 0, 0,
                   2, 0, 0, 0, 1, 0, 0,
                   0, 2, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected: 
# 1 result is 4
# 9 result is 3
# 17 result is 2
# 25 result is 1
# 5 result is 2 # test edge
print('Test with positive argument for increment by 8')
for i in [1, 9, 17, 25, 5]:
    print('Parameter: {} - value: {}'.format(i, cb.count_from(i, increment=8)))
    
print('Test with negative argument for increment by -8 - decrement')
# Expected: 
# 1 result is 1
# 9 result is 2
# 17 result is 3
# 25 result is 4
# 29 result is 2 # test edge
for i in [1, 9, 17, 25, 29]:
    print('Parameter: {} - value: {}'.format(i, cb.count_from(i, increment=-8)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  | 2|  |  |  |  |  |
----------------------
| 2|  |  |  | 1|  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------
|  |  | 1|  |  |  | 2|
----------------------
|  | 1|  |  |  | 2|  |
----------------------
Test with positive argument for increment by 8
Parameter: 1 - value: 4
Parameter: 9 - value: 3
Parameter: 17 - value: 2
Parameter: 25 - value: 1
Parameter: 5 - value: 2
Test with negative argument for increment by -8 - decrement
Parameter: 1 - value: 1
Parameter: 9 - value: 2
Parameter: 17 - value: 3
Parameter: 25 - value: 4
Parameter: 29 - value: 2


### Test count_from() - diagonal left up

In [21]:
board = np.array([ 0, 1, 0, 0, 1, 0, 0,
                   1, 0, 0, 1, 0, 0, 1,
                   0, 0, 1, 0, 0, 1, 0,
                   0, 1, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected:
# 4 result is 4
# 10 result is 3
# 16 result is 2
# 22 result is 1
# 1 result is 2 # test edge

print('Test with positive argument for increment by 6')
for i in [4, 10, 16, 22, 1]:
    print('Parameter: {} - value: {}'.format(i, cb.count_from(i, increment=6)))
    
print('Test with negative argument for increment by -6 - decrement')
# Expected:
# 4 result is 1
# 10 result is 2
# 16 result is 3
# 22 result is 4
# 19 result is 2 # test edge
for i in [4, 10, 16, 22, 13]:
    print('Parameter: {} - value: {}'.format(i, cb.count_from(i, increment=-6)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  | 1|  |  |  |  |  |
----------------------
|  |  | 1|  |  | 1|  |
----------------------
| 1|  |  | 1|  |  | 1|
----------------------
|  | 1|  |  | 1|  |  |
----------------------
Test with positive argument for increment by 6
Parameter: 4 - value: 4
Parameter: 10 - value: 3
Parameter: 16 - value: 2
Parameter: 22 - value: 1
Parameter: 1 - value: 2
Test with negative argument for increment by -6 - decrement
Parameter: 4 - value: 1
Parameter: 10 - value: 2
Parameter: 16 - value: 3
Parameter: 22 - value: 4
Parameter: 13 - value: 1


## Test game_won_at(index)

### Test game_won_at - horizontally

In [22]:
board = np.array([ 1, 1, 1, 1, 0, 2, 2,
                   2, 2, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected: 
# 0 result is True
# 1 result is True
# 2 result is True
# 3 result is True
# 5 result is False # test edge
print('Test with positive argument for increment')
for i in [0, 1, 2, 3, 5]:
    print('Parameter: {} - value: {}'.format(i, cb.game_won_at(i)))
    
print('Test with negative argument for increment')
# Expected: 
# 0 result is True
# 1 result is True
# 2 result is True
# 3 result is True
# 8 result is False # test edge
for i in [0, 1, 2, 3, 8]:
    print('Parameter: {} - value: {}'.format(i, cb.game_won_at(i)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
| 2| 2|  |  |  |  |  |
----------------------
| 1| 1| 1| 1|  | 2| 2|
----------------------
Test with positive argument for increment
Parameter: 0 - value: True
Parameter: 1 - value: True
Parameter: 2 - value: True
Parameter: 3 - value: True
Parameter: 5 - value: False
Test with negative argument for increment
Parameter: 0 - value: True
Parameter: 1 - value: True
Parameter: 2 - value: True
Parameter: 3 - value: True
Parameter: 8 - value: False


### Test game_won_at - vertical

In [23]:
board = np.array([ 0, 0, 0, 0, 0, 0, 2,
                   0, 0, 0, 0, 0, 0, 2,
                   0, 0, 0, 0, 0, 0, 2,
                   0, 0, 0, 0, 0, 0, 2,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected: 
# 6 result is True
# 13 result is True
# 20 result is True
# 27 result is True
print('Test with positive argument for increment by 7')
for i in [6, 13, 20, 27]:
    print('Parameter: {} - value: {}'.format(i, cb.game_won_at(i)))
    
print('Test with negative argument for increment by -7 - decrement')
# Expected: 
# 6 result is True
# 13 result is True
# 20 result is True
# 27 result is True
for i in [6, 13, 20, 27]:
    print('Parameter: {} - value: {}'.format(i, cb.game_won_at(i)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  | 2|
----------------------
|  |  |  |  |  |  | 2|
----------------------
|  |  |  |  |  |  | 2|
----------------------
|  |  |  |  |  |  | 2|
----------------------
Test with positive argument for increment by 7
Parameter: 6 - value: True
Parameter: 13 - value: True
Parameter: 20 - value: True
Parameter: 27 - value: True
Test with negative argument for increment by -7 - decrement
Parameter: 6 - value: True
Parameter: 13 - value: True
Parameter: 20 - value: True
Parameter: 27 - value: True


### Test game_won_at - diagonal right up

In [24]:
board = np.array([ 0, 1, 0, 0, 0, 2, 0,
                   0, 0, 1, 0, 0, 0, 2,
                   0, 0, 0, 1, 0, 0, 0,
                   2, 0, 0, 0, 1, 0, 0,
                   0, 2, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected: 
# 1 result is True
# 9 result is True
# 17 result is True
# 25 result is True
# 5 result is False # test edge
print('Test with positive argument for increment by 8')
for i in [1, 9, 17, 25, 5]:
    print('Parameter: {} - value: {}'.format(i, cb.game_won_at(i)))
    
print('Test with negative argument for increment by -8 - decrement')
# Expected: 
# 1 result is True
# 9 result is True
# 17 result is True
# 25 result is True
# 29 result is False # test edge
for i in [1, 9, 17, 25, 29]:
    print('Parameter: {} - value: {}'.format(i, cb.game_won_at(i)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  | 2|  |  |  |  |  |
----------------------
| 2|  |  |  | 1|  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------
|  |  | 1|  |  |  | 2|
----------------------
|  | 1|  |  |  | 2|  |
----------------------
Test with positive argument for increment by 8
Parameter: 1 - value: True
Parameter: 9 - value: True
Parameter: 17 - value: True
Parameter: 25 - value: True
Parameter: 5 - value: False
Test with negative argument for increment by -8 - decrement
Parameter: 1 - value: True
Parameter: 9 - value: True
Parameter: 17 - value: True
Parameter: 25 - value: True
Parameter: 29 - value: False


### Test game_won_at - diagonal left up

In [25]:
board = np.array([ 0, 1, 0, 0, 1, 0, 0,
                   1, 0, 0, 1, 0, 0, 1,
                   0, 0, 1, 0, 0, 1, 0,
                   0, 1, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected:
# 4 result is True
# 10 result is True
# 16 result is True
# 22 result is True
# 1 result is False # test edge

print('Test with positive argument for increment by 6')
for i in [4, 10, 16, 22, 1]:
    print('Parameter: {} - value: {}'.format(i, cb.game_won_at(i)))
    
print('Test with negative argument for increment by -6 - decrement')
# Expected:
# 4 result is True
# 10 result is True
# 16 result is True
# 22 result is True
# 19 result is False # test edge
for i in [4, 10, 16, 22, 13]:
    print('Parameter: {} - value: {}'.format(i, cb.game_won_at(i)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  | 1|  |  |  |  |  |
----------------------
|  |  | 1|  |  | 1|  |
----------------------
| 1|  |  | 1|  |  | 1|
----------------------
|  | 1|  |  | 1|  |  |
----------------------
Test with positive argument for increment by 6
Parameter: 4 - value: True
Parameter: 10 - value: True
Parameter: 16 - value: True
Parameter: 22 - value: True
Parameter: 1 - value: False
Test with negative argument for increment by -6 - decrement
Parameter: 4 - value: True
Parameter: 10 - value: True
Parameter: 16 - value: True
Parameter: 22 - value: True
Parameter: 13 - value: False


### Test game_won() and outcome
- outcome should be equal the result of game_won()

In [26]:
board=np.array([ 1, 2, 0, 0, 0, 0, 1,
                 1, 0, 2, 0, 0, 1, 0,
                 1, 0, 0, 2, 1, 0, 0,
                 1, 0, 0, 1, 2, 0, 0,
                 0, 0, 0, 0, 0, 0, 0,
                 2, 2, 2, 2, 0, 0, 1], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected:
# 0 result: 9223372036854775807; outcome: 9223372036854775807
# 1 result: 9223372036854775807; outcome: 9223372036854775807
# 6 result: 9223372036854775807; outcome: 9223372036854775807
# 35 result: 9223372036854775807; outcome: 9223372036854775807
# 41 result: #TODO some number but not 9223372036854775807; outcome: #TODO some number but not 9223372036854775807 # test edge

for i in [0, 1, 6, 35, 41]:
    cb = CurrentBoard(board, index=i)
    print('parameter value: {}; result: {}; outcome: {}'.format(i, cb.game_won(i), cb.outcome))

| 1| 2| 3| 4| 5| 6| 7|
| 2| 2| 2| 2|  |  | 1|
----------------------
|  |  |  |  |  |  |  |
----------------------
| 1|  |  | 1| 2|  |  |
----------------------
| 1|  |  | 2| 1|  |  |
----------------------
| 1|  | 2|  |  | 1|  |
----------------------
| 1| 2|  |  |  |  | 1|
----------------------
parameter value: 0; result: 9223372036854775807; outcome: 9223372036854775807
parameter value: 1; result: 9223372036854775807; outcome: 9223372036854775807
parameter value: 6; result: 9223372036854775807; outcome: 9223372036854775807
parameter value: 35; result: 9223372036854775807; outcome: 9223372036854775807
parameter value: 41; result: 6; outcome: 3


# TODO

## eval_count_from(self, start_index, increment, start_value)

### eval_count_from - horizontally

In [27]:
board = np.array([ 1, 0, 0, 0, 0, 0, 1,
                   1, 2, 0, 0, 0, 2, 1,
                   1, 0, 0, 1, 0, 0, 1,
                   1, 1, 0, 0, 0, 1, 1,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected: 
# 0 result is 6
# 7 result is 6
# 14 result is 3
# 21 result is 1

print('Test with positive argument for increment')
for i in [0, 7, 14, 21]:
    print('Parameter: {} - value: {}'.format(i, cb.eval_count_from(start_index=i, increment=1, start_value=1)))
    
print('Test with negative argument for increment')
# Expected:
# 6 result is 6
# 13 result is 6
# 20 result is 3
# 27 result is 1
for i in [6, 13, 20, 27]:
    print('Parameter: {} - value: {}'.format(i, cb.eval_count_from(start_index=i, increment=-1, start_value=1)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
| 1| 1|  |  |  | 1| 1|
----------------------
| 1|  |  | 1|  |  | 1|
----------------------
| 1| 2|  |  |  | 2| 1|
----------------------
| 1|  |  |  |  |  | 1|
----------------------
Test with positive argument for increment
Parameter: 0 - value: 6
Parameter: 7 - value: 6
Parameter: 14 - value: 3
Parameter: 21 - value: 1
Test with negative argument for increment
Parameter: 6 - value: 6
Parameter: 13 - value: 6
Parameter: 20 - value: 3
Parameter: 27 - value: 1


### eval_count_from - verticaly

### eval_count_from - diagonaly right up/left dow

### eval_count_from - diagonaly left up/right down

## Test total_blocked_positions(self, index)

### Test total_blocked_positions(self, index)

In [28]:
board = np.array([ 0, 0, 0, 1, 0, 0, 0,
                   1, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 0, 0,
                   0, 0, 0, 0, 0, 1, 1,
                   0, 0, 0, 0, 0, 1, 1], dtype=int)
cb = CurrentBoard(board)
cb.display_game_board()
# Expected: 
# 3 result is 7
# 7 result is 4
# 41 result is 0

for i in [3, 7, 41]:
    print('Parameter: {} - value: {}'.format(i, cb.total_blocked_positions(i)))

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  | 1| 1|
----------------------
|  |  |  |  |  | 1| 1|
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
| 1|  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------
Parameter: 3 - value: 7
Parameter: 7 - value: 4
Parameter: 41 - value: 0


# TEST SearchTreeNode CLASS

## Tests SearchTreeNode object creation

In all cases 'nex_move' is None, as next_move is assign by min_max method

### Test 1) Start of the game, default board
Create default SearchTreeNode with max_depth=3 and check all levels. Last level should not create children.

In [29]:
# default SearchTreeNode
stn = SearchTreeNode(max_depth=3)

In [30]:
# Expected: empty board
# 'count_move': 0,
# 'index': -1,
# 'player_at_this_level': 2,
# 'player_1': 1,
# 'player_2': 2,
# 'win': False,
# 'outcome': 0

# level 0
level_0 = stn
level_0.game_at_this_node.display_game_board()
print('CurrentBoard at this node properties:')
vars(level_0.game_at_this_node)

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
CurrentBoard at this node properties:


{'board': array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'board_width': 7,
 'board_height': 6,
 'board_total': 42,
 'count_move': 0,
 'index': -1,
 'player_at_this_level': 2,
 'winning_positions_1': 69,
 'winning_positions_2': 69,
 'player_1': 1,
 'player_2': 2,
 'win': False,
 'draw': 0,
 'outcome': 0}

In [31]:
# level 0
print('SearchTreeNode properties:')
vars(level_0)

SearchTreeNode properties:


{'children': [<__main__.SearchTreeNode at 0x22ad9b0be10>,
 'ply_depth': 0,
 'max_depth': 3,
 'node_terminal': False,
 'node_trap': False,
 'node_value': None,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9ba1f28>,
 'comp_first': True}

In [32]:
# Check if items in children are sorted in descending order accordind to heuristic function (outcome)
for child in level_0.children:
    print(child.game_at_this_node.outcome, end=' ')

7 5 4 3 

In [33]:
# level 1
level_1 = stn.children[0]
level_1.game_at_this_node.display_game_board()
print('CurrentBoard at this node properties:')
vars(level_1.game_at_this_node)

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------
CurrentBoard at this node properties:


{'board': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'board_width': 7,
 'board_height': 6,
 'board_total': 42,
 'count_move': 1,
 'index': 3,
 'player_at_this_level': 1,
 'winning_positions_1': 69,
 'winning_positions_2': 62,
 'player_1': 1,
 'player_2': 2,
 'win': False,
 'draw': 0,
 'outcome': 7}

In [34]:
print('SearchTreeNode properties:')
vars(level_1)

SearchTreeNode properties:


{'children': [<__main__.SearchTreeNode at 0x22ad9b32f28>,
 'ply_depth': 1,
 'max_depth': 3,
 'node_terminal': False,
 'node_trap': False,
 'node_value': None,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9ba17f0>,
 'comp_first': True}

In [35]:
# Check if items in children are sorted in descending order accordind to heuristic function (outcome)
for child in level_1.children:
    print(child.game_at_this_node.outcome, end=' ')

10 5 4 3 

In [36]:
# level 2
level_2 = stn.children[0].children[0]
level_2.game_at_this_node.display_game_board()
print('CurrentBoard at this node properties:')
vars(level_2.game_at_this_node)

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 2|  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------
CurrentBoard at this node properties:


{'board': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'board_width': 7,
 'board_height': 6,
 'board_total': 42,
 'count_move': 2,
 'index': 10,
 'player_at_this_level': 2,
 'winning_positions_1': 59,
 'winning_positions_2': 69,
 'player_1': 1,
 'player_2': 2,
 'win': False,
 'draw': 0,
 'outcome': 10}

In [37]:
print('SearchTreeNode properties:')
vars(level_2)

SearchTreeNode properties:


{'children': [<__main__.SearchTreeNode at 0x22ad9b0a2e8>,
 'ply_depth': 2,
 'max_depth': 3,
 'node_terminal': False,
 'node_trap': False,
 'node_value': None,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b32668>,
 'comp_first': True}

In [38]:
# Expected:
# 'count_move': 3,
# 'index': 17,
# 'player_at_this_level': 1,
# 'player_1': 1,
# 'player_2': 2,
# 'win': False,
# 'outcome': 3 (REMEMBER: outcome depends on eval function)

# level 3
level_3 = stn.children[0].children[0].children[0]
level_3.game_at_this_node.display_game_board()
print('CurrentBoard at this node properties:')
vars(level_3.game_at_this_node)

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------
|  |  |  | 2|  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------
CurrentBoard at this node properties:


{'board': array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 'board_width': 7,
 'board_height': 6,
 'board_total': 42,
 'count_move': 3,
 'index': 17,
 'player_at_this_level': 1,
 'winning_positions_1': 69,
 'winning_positions_2': 57,
 'player_1': 1,
 'player_2': 2,
 'win': False,
 'draw': 0,
 'outcome': 12}

In [39]:
# Case: ply_depth == max_depth
# Expected: 
# 'children': [],
# 'ply_depth': 3,
# 'max_depth': 3,
# 'node_value': None,
# 'game_at_this_node': <__main__.CurrentBoard >,
# 'comp_first': True,
print('SearchTreeNode properties:')
vars(level_3)

SearchTreeNode properties:


{'children': [],
 'ply_depth': 3,
 'max_depth': 3,
 'node_terminal': False,
 'node_trap': False,
 'node_value': None,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b0a1d0>,
 'comp_first': True}

In [40]:
# Check if items in children are sorted in descending order accordind to heuristic function (outcome)
for child in level_1.children:
    print(child.game_at_this_node.outcome, end=' ')

10 5 4 3 

### Test 2) win situation for player 1

In [41]:
# board filled with: 1, 2 and few available moves
board=np.array([2, 2, 2, 1, 1, 2, 2,
                1, 2, 1, 2, 1, 1, 1,
                2, 2, 2, 1, 1, 2, 1,
                2, 1, 2, 1, 2, 2, 1,
                2, 1, 2, 1, 1, 1, 2,
                1, 2, 1, 0, 0, 1, 2], dtype=int)
cb = CurrentBoard(board, count_move=40, index=41, player_at_this_level=2)
stn = SearchTreeNode(Current_Game=cb, max_depth=7)

stn.game_at_this_node.display_game_board()
print('CurrentBoard at this node properties:')
vars(stn)

| 1| 2| 3| 4| 5| 6| 7|
| 1| 2| 1|  |  | 1| 2|
----------------------
| 2| 1| 2| 1| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 2| 2| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 1|
----------------------
| 1| 2| 1| 2| 1| 1| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 2|
----------------------
CurrentBoard at this node properties:


{'children': [<__main__.SearchTreeNode at 0x22ad9b316d8>,
 'ply_depth': 0,
 'max_depth': 7,
 'node_terminal': False,
 'node_trap': False,
 'node_value': None,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b31588>,
 'comp_first': True}

In [42]:
# Expected: one of the child should have: 
# children': []
# node_terminal': True


print('SearchTreeNode properties of child 0:')
print(vars(stn.children[0]))
print('SearchTreeNode properties of child 1:')
print(vars(stn.children[1]))

SearchTreeNode properties of child 0:
{'children': [], 'ply_depth': 1, 'max_depth': 7, 'node_terminal': True, 'node_trap': False, 'node_value': None, 'game_at_this_node': <__main__.CurrentBoard object at 0x0000022AD9B31668>, 'comp_first': True}
SearchTreeNode properties of child 1:
{'children': [<__main__.SearchTreeNode object at 0x0000022AD9B317B8>], 'ply_depth': 1, 'max_depth': 7, 'node_terminal': False, 'node_trap': False, 'node_value': None, 'game_at_this_node': <__main__.CurrentBoard object at 0x0000022AD9B315F8>, 'comp_first': True}


### Test 3) win situation for player 2

In [43]:
# board filled with: 1, 2 and few available moves
board=np.array([2, 2, 2, 1, 1, 2, 2,
                1, 2, 1, 2, 1, 1, 1,
                2, 2, 2, 1, 1, 2, 1,
                2, 1, 2, 1, 2, 2, 1,
                2, 1, 2, 1, 1, 1, 2,
                1, 2, 0, 2, 1, 1, 2], dtype=int)
cb = CurrentBoard(board, count_move=41, index=41, player_at_this_level=1)
stn = SearchTreeNode(Current_Game=cb, max_depth=7)

stn.game_at_this_node.display_game_board()
print('CurrentBoard at this node properties:')
vars(stn)

| 1| 2| 3| 4| 5| 6| 7|
| 1| 2|  | 2| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 2| 2| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 1|
----------------------
| 1| 2| 1| 2| 1| 1| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 2|
----------------------
CurrentBoard at this node properties:


{'children': [<__main__.SearchTreeNode at 0x22ad9b31fd0>],
 'ply_depth': 0,
 'max_depth': 7,
 'node_terminal': False,
 'node_trap': False,
 'node_value': None,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b31e10>,
 'comp_first': True}

In [44]:
# Expected:
# children': []
# 'node_terminal': True,
# 'node_value': None,

vars(stn.children[0])

{'children': [],
 'ply_depth': 1,
 'max_depth': 7,
 'node_terminal': True,
 'node_trap': False,
 'node_value': None,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b31f60>,
 'comp_first': True}

### Test 4) draw situation

In [45]:
# board filled with: 1, 2 and few available moves
board=np.array([2, 2, 2, 1, 1, 2, 2,
                1, 2, 1, 2, 1, 1, 1,
                2, 2, 2, 1, 1, 2, 1,
                2, 1, 2, 1, 2, 2, 1,
                2, 1, 2, 1, 1, 1, 2,
                1, 2, 1, 2, 0, 1, 2], dtype=int)
cb = CurrentBoard(board, count_move=41, index=41, player_at_this_level=1)
stn = SearchTreeNode(Current_Game=cb, max_depth=7)

stn.game_at_this_node.display_game_board()
print('CurrentBoard at this node properties:')
vars(stn)

| 1| 2| 3| 4| 5| 6| 7|
| 1| 2| 1| 2|  | 1| 2|
----------------------
| 2| 1| 2| 1| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 2| 2| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 1|
----------------------
| 1| 2| 1| 2| 1| 1| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 2|
----------------------
CurrentBoard at this node properties:


{'children': [<__main__.SearchTreeNode at 0x22ad9b516a0>],
 'ply_depth': 0,
 'max_depth': 7,
 'node_terminal': False,
 'node_trap': False,
 'node_value': None,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b51550>,
 'comp_first': True}

In [46]:
# Expected:
# children': []
# 'node_terminal': True,
# 'node_value': None,

vars(stn.children[0])

{'children': [],
 'ply_depth': 1,
 'max_depth': 7,
 'node_terminal': True,
 'node_trap': False,
 'node_value': None,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b51668>,
 'comp_first': True}

## TEST min_max() and get_next_move()

### Test 1) Default SearchTreeNode
### Test 1a) min_max()

In [47]:
stn = SearchTreeNode(max_depth=3) # default SearchTreeNode with max_depth=3
stn.min_max()
# Expected:
# 'node_terminal': False,
# 'node_value': 10, (may be different if the eval function is changed)

vars(stn)

{'children': [<__main__.SearchTreeNode at 0x22ad9b2b278>,
 'ply_depth': 0,
 'max_depth': 3,
 'node_terminal': False,
 'node_trap': False,
 'node_value': 9,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b314e0>,
 'comp_first': True}

In [48]:
# Test the max layer
# Expected:
# the node_value of a parent node and node_value of one of his child should have equal value
# and the node_value of that child should be the max node_value of all children

# 'node_terminal': False,
# 'node_value': 10, (may be different if the eval function is changed) - the same value as stn.node_value
print('PARENT: node_terminal: {}; node_value: {}'.format(stn.node_terminal, stn.node_value))

for child in stn.children:
    print('CHILD: node_terminal: {}; node_value: {}'.format(child.node_terminal, child.node_value))

PARENT: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 7
CHILD: node_terminal: False; node_value: 5
CHILD: node_terminal: False; node_value: 6


In [49]:
# Test the min layer
# Expected:
# the node_value of a parent node and node_value of one of his child should have equal value
# and the node_value of that child should be the min node_value of all children

# 'node_terminal': False,
# 'node_value': 10, (may be different if the eval function is changed) - the same value as stn.node_value

for parent in stn.children:
    if parent.node_value == stn.node_value:
        print('PARENT: node_terminal: {}; node_value: {}'.format(parent.node_terminal, parent.node_value))
        for child in parent.children:
            print('CHILD: node_terminal: {}; node_value: {}'.format(child.node_terminal, child.node_value))


PARENT: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 12
CHILD: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 9


### Test 1b) get_next_move()

In [50]:
# Expected: the SearchTreeNode
vars(stn.get_next_move())

{'children': [<__main__.SearchTreeNode at 0x22ad9b3f390>,
 'ply_depth': 1,
 'max_depth': 3,
 'node_terminal': False,
 'node_trap': False,
 'node_value': 9,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b31a90>,
 'comp_first': True}

In [51]:
# Expected: True
stn.node_value == stn.get_next_move().node_value 

True

In [52]:
# Expected: the first move in the middle of the bottom row. It is the only position that guarantees winning of a game
stn.get_next_move().game_at_this_node.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------


### Test 2) Win situation for player 1
- index is from 0 to 41, count_move is from 0 to 42 including 42.
- player_1 makes first move, meaning that player_1 is on odd count_move, player_2 is on even count_move
### Test 2a) min_max()
- the root node is a max layer


In [53]:
# board filled with: 1, 2 and few available moves
board=np.array([2, 2, 2, 1, 1, 2, 2,
                1, 2, 1, 2, 1, 1, 1,
                2, 2, 2, 1, 1, 2, 1,
                2, 1, 2, 1, 2, 2, 1,
                2, 1, 2, 1, 1, 1, 2,
                1, 2, 1, 0, 0, 1, 2], dtype=int)
cb = CurrentBoard(board, count_move=40, index=41, player_at_this_level=2)
stn = SearchTreeNode(Current_Game=cb)
stn.game_at_this_node.display_game_board()

stn.min_max()
# Expected:
# 'node_terminal': False,
# 'node_value': 9223372036854775807

vars(stn)

| 1| 2| 3| 4| 5| 6| 7|
| 1| 2| 1|  |  | 1| 2|
----------------------
| 2| 1| 2| 1| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 2| 2| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 1|
----------------------
| 1| 2| 1| 2| 1| 1| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 2|
----------------------


{'children': [<__main__.SearchTreeNode at 0x22ad9bdcba8>,
 'ply_depth': 0,
 'max_depth': 4,
 'node_terminal': False,
 'node_trap': False,
 'node_value': 9223372036854775807,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9bdca58>,
 'comp_first': True}

In [54]:
# Test the max layer
# Expected:
# the node_value of a parent node and node_value of one of his child should have equal value
# and the node_value of that child should be the max node_value of all children
# and the node_terminal of that child should be True

# 'node_terminal': False,
# 'node_value': 9223372036854775807, (may be different if the eval function is changed) - the same value as stn.node_value
print('PARENT: node_terminal: {}; node_value: {}'.format(stn.node_terminal, stn.node_value))

for child in stn.children:
    print('CHILD: node_terminal: {}; node_value: {}'.format(child.node_terminal, child.node_value))

PARENT: node_terminal: False; node_value: 9223372036854775807
CHILD: node_terminal: True; node_value: 9223372036854775807
CHILD: node_terminal: False; node_value: 0


### Test 2b) get_next_move()

In [55]:
# Expected: the SearchTreeNode
# 'node_terminal': True,
# 'node_value': 9223372036854775807,
vars(stn.get_next_move())

{'children': [],
 'ply_depth': 1,
 'max_depth': 4,
 'node_terminal': True,
 'node_trap': False,
 'node_value': 9223372036854775807,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9bdcb38>,
 'comp_first': True}

In [56]:
# Expected: True
stn.node_value == stn.get_next_move().node_value 

True

### Test 3) Lose situation for player 1 / Win situation for player 2
- index is from 0 to 41, count_move is from 0 to 42 including 42.
- player_1 makes first move, meaning that player_1 is on odd count_move, player_2 is on even count_move
### Test 3a) min_max()
- the root node is a min layer

In [57]:
board=np.array([2, 2, 2, 1, 1, 2, 2,
                1, 2, 1, 2, 1, 1, 1,
                2, 2, 2, 1, 1, 2, 1,
                2, 1, 2, 1, 2, 2, 1,
                2, 1, 2, 1, 1, 1, 2,
                1, 2, 0, 2, 0, 0, 1], dtype=int)
cb = CurrentBoard(board, count_move=39, index=41, player_at_this_level=1)
stn = SearchTreeNode(Current_Game=cb)
stn.game_at_this_node.display_game_board()

stn.min_max(is_max=False)
# Expected:
# 'node_terminal': False,
# 'node_value': -9223372036854775807

vars(stn)

| 1| 2| 3| 4| 5| 6| 7|
| 1| 2|  | 2|  |  | 1|
----------------------
| 2| 1| 2| 1| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 2| 2| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 1|
----------------------
| 1| 2| 1| 2| 1| 1| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 2|
----------------------


{'children': [<__main__.SearchTreeNode at 0x22ad9be2ac8>,
 'ply_depth': 0,
 'max_depth': 4,
 'node_terminal': False,
 'node_trap': False,
 'node_value': -9223372036854775807,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9be2940>,
 'comp_first': True}

In [58]:
# Test the min layer
# Expected:
# the node_value of a parent node and node_value of one of his child should have equal value
# and the node_value of that child should be the min node_value of all children
# and the node_terminal of that child should be True

# 'node_terminal': False,
# 'node_value': -9223372036854775807, (may be different if the eval function is changed) - the same value as stn.node_value
print('PARENT: node_terminal: {}; node_value: {}'.format(stn.node_terminal, stn.node_value))

for child in stn.children:
    print('CHILD: node_terminal: {}; node_value: {}'.format(child.node_terminal, child.node_value))

PARENT: node_terminal: False; node_value: -9223372036854775807
CHILD: node_terminal: True; node_value: -9223372036854775807
CHILD: node_terminal: False; node_value: 0
CHILD: node_terminal: False; node_value: 0


### Test 3b) get_next_move()

In [59]:
# Expected: the SearchTreeNode
# 'node_terminal': True,
# 'node_value': -9223372036854775807,
vars(stn.get_next_move())

{'children': [],
 'ply_depth': 1,
 'max_depth': 4,
 'node_terminal': True,
 'node_trap': False,
 'node_value': -9223372036854775807,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9be2a20>,
 'comp_first': True}

In [60]:
# Expected: True
stn.node_value == stn.get_next_move().node_value 

True

### Test 4) draw situation
- index is from 0 to 41, count_move is from 0 to 42 including 42.
- player_1 makes first move, meaning that player_1 is on odd count_move, player_2 is on even count_move
### Test 4a) min_max()
- the root node is a max layer

In [61]:
board = np.array([2, 2, 2, 1, 1, 2, 2,
                    1, 2, 1, 2, 1, 1, 1,
                    2, 2, 2, 1, 1, 2, 1,
                    2, 1, 2, 1, 2, 2, 1,
                    2, 1, 2, 1, 1, 1, 2,
                    1, 2, 1, 2, 0, 1, 2], dtype=int)

cb = CurrentBoard(setup_of_board=board, count_move=41, index=40 , player_at_this_level=1)
stn = SearchTreeNode(Current_Game=cb)
stn.game_at_this_node.display_game_board()

stn.min_max(is_max=False)
# Expected:
# 'node_terminal': False,
# 'node_value': 0

vars(stn)

| 1| 2| 3| 4| 5| 6| 7|
| 1| 2| 1| 2|  | 1| 2|
----------------------
| 2| 1| 2| 1| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 2| 2| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 1|
----------------------
| 1| 2| 1| 2| 1| 1| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 2|
----------------------


{'children': [<__main__.SearchTreeNode at 0x22ad9b79ba8>],
 'ply_depth': 0,
 'max_depth': 4,
 'node_terminal': False,
 'node_trap': False,
 'node_value': 0,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b79a90>,
 'comp_first': True}

In [62]:
# Test the max layer
# Expected:
# the node_value of a parent node and node_value of one of his child should have equal value
# and the node_value of that child should be the max node_value of all children
# and the node_terminal of that child should be True

# 'node_terminal': False,
# 'node_value': 0, (may be different if the eval function is changed) - the same value as stn.node_value
print('PARENT: node_terminal: {}; node_value: {}'.format(stn.node_terminal, stn.node_value))

for child in stn.children:
    print('CHILD: node_terminal: {}; node_value: {}'.format(child.node_terminal, child.node_value))

PARENT: node_terminal: False; node_value: 0
CHILD: node_terminal: True; node_value: 0


### Test 4b) get_next_move()

In [63]:
# Expected: the SearchTreeNode
# 'node_terminal': True,
# 'node_value': 0,
vars(stn.get_next_move())

{'children': [],
 'ply_depth': 1,
 'max_depth': 4,
 'node_terminal': True,
 'node_trap': False,
 'node_value': 0,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b79b70>,
 'comp_first': True}

In [64]:
# Expected: True
stn.node_value == stn.get_next_move().node_value 

True

# Test min_max_alpha_beta()

Check the tree_size before and after running max_min_alpha_beta().
Expected: the tree size should be smaller after running min_max_alpha_beta()

### Test 1) Default SearchTreeNode
### Test 1a) min_max_alpha_beta()

In [65]:
stn = SearchTreeNode(max_depth=3) # default SearchTreeNode with max_depth=3
size_before = stn.tree_size()

In [66]:
stn.min_max_alpha_beta()

size_after = stn.tree_size()
# Expected:
# 'node_terminal': False,
# 'node_value': 10, (may be different if the eval function is changed)

vars(stn)

{'children': [<__main__.SearchTreeNode at 0x22ad9bf0e10>,
 'ply_depth': 0,
 'max_depth': 3,
 'node_terminal': False,
 'node_trap': False,
 'node_value': 9,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b79668>,
 'comp_first': True}

In [67]:
# Test the max layer
# Expected:
# the node_value of a parent node and node_value of one of his child should have equal value
# and the node_value of that child should be the max node_value of all children

# 'node_terminal': False,
# 'node_value':  (may be different if the eval function is changed) - the same value as stn.node_value
print('PARENT: node_terminal: {}; node_value: {}'.format(stn.node_terminal, stn.node_value))

for child in stn.children:
    print('CHILD: node_terminal: {}; node_value: {}'.format(child.node_terminal, child.node_value))

PARENT: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 7
CHILD: node_terminal: False; node_value: 8


In [68]:
# Test the min layer
# Expected:
# the node_value of a parent node and node_value of one of his child should have equal value
# and the node_value of that child should be the min node_value of all children

# 'node_terminal': False,
# 'node_value':  (may be different if the eval function is changed) - the same value as stn.node_value

for parent in stn.children:
    if parent.node_value == stn.node_value:
        print('PARENT: node_terminal: {}; node_value: {}'.format(parent.node_terminal, parent.node_value))
        for child in parent.children:
            print('CHILD: node_terminal: {}; node_value: {}'.format(child.node_terminal, child.node_value))


PARENT: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 12
CHILD: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 9
PARENT: node_terminal: False; node_value: 9
CHILD: node_terminal: False; node_value: 10
CHILD: node_terminal: False; node_value: 9


### Test 1a.a) Check the size befor and after pruning

In [69]:
#Expected: True
print('size_before: {}; size_after: {}'.format(size_before, size_after))
size_before > size_after

size_before: 201; size_after: 69


True

### Test 1b) get_next_move()

In [70]:
# Expected: the SearchTreeNode
vars(stn.get_next_move())

{'children': [<__main__.SearchTreeNode at 0x22ad9bfaf28>,
 'ply_depth': 1,
 'max_depth': 3,
 'node_terminal': False,
 'node_trap': False,
 'node_value': 9,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b79780>,
 'comp_first': True}

In [71]:
# Expected: True
stn.node_value == stn.get_next_move().node_value 

True

In [72]:
# Expected: the first move in the middle of the bottom row. It is the only position that guarantees winning of a game
stn.get_next_move().game_at_this_node.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------


### Test 2) Win situation for player 1
- index is from 0 to 41, count_move is from 0 to 42 including 42.
- player_1 makes first move, meaning that player_1 is on odd count_move, player_2 is on even count_move
### Test 2a) min_max_alpha_beta()
- the root node is a max layer


In [73]:
# board filled with: 1, 2 and few available moves
board=np.array([2, 2, 2, 1, 1, 2, 2,
                1, 2, 1, 2, 1, 1, 1,
                2, 2, 2, 1, 1, 2, 1,
                2, 1, 2, 1, 2, 2, 1,
                2, 1, 2, 1, 1, 1, 2,
                1, 2, 1, 0, 0, 1, 2], dtype=int)
cb = CurrentBoard(board, count_move=40, index=41, player_at_this_level=2)
stn = SearchTreeNode(Current_Game=cb)
stn.game_at_this_node.display_game_board()

stn.min_max_alpha_beta()
# Expected:
# 'node_terminal': False,
# 'node_value': 9223372036854775807

vars(stn)

| 1| 2| 3| 4| 5| 6| 7|
| 1| 2| 1|  |  | 1| 2|
----------------------
| 2| 1| 2| 1| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 2| 2| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 1|
----------------------
| 1| 2| 1| 2| 1| 1| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 2|
----------------------


{'children': [<__main__.SearchTreeNode at 0x22ad9b07be0>],
 'ply_depth': 0,
 'max_depth': 4,
 'node_terminal': False,
 'node_trap': False,
 'node_value': 9223372036854775807,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9ba1da0>,
 'comp_first': True}

In [74]:
# Test the max layer
# Expected:
# the node_value of a parent node and node_value of one of his child should have equal value
# and the node_value of that child should be the max node_value of all children
# and the node_terminal of that child should be True

# 'node_terminal': False,
# 'node_value': 9223372036854775807, (may be different if the eval function is changed) - the same value as stn.node_value
print('PARENT: node_terminal: {}; node_value: {}'.format(stn.node_terminal, stn.node_value))

# 'node_terminal': True,
# 'node_value': 9223372036854775807, (may be different if the eval function is changed) - the same value as stn.node_value
for child in stn.children:
    print('CHILD: node_terminal: {}; node_value: {}'.format(child.node_terminal, child.node_value))

PARENT: node_terminal: False; node_value: 9223372036854775807
CHILD: node_terminal: True; node_value: 9223372036854775807


### Test 2b) get_next_move()

In [75]:
# Expected: the SearchTreeNode
# 'node_terminal': True,
# 'node_value': 9223372036854775807,
vars(stn.get_next_move())

{'children': [],
 'ply_depth': 1,
 'max_depth': 4,
 'node_terminal': True,
 'node_trap': False,
 'node_value': 9223372036854775807,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b075c0>,
 'comp_first': True}

In [76]:
# Expected: True
stn.node_value == stn.get_next_move().node_value 

True

### Test 3) Lose situation for player 1 / Win situation for player 2
- index is from 0 to 41, count_move is from 0 to 42 including 42.
- player_1 makes first move, meaning that player_1 is on odd count_move, player_2 is on even count_move
### Test 3a) min_max_alpha_beta()
- the root node is a min layer

In [77]:
board=np.array([2, 2, 2, 1, 1, 2, 2,
                1, 2, 1, 2, 1, 1, 1,
                2, 2, 2, 1, 1, 2, 1,
                2, 1, 2, 1, 2, 2, 1,
                2, 1, 2, 1, 1, 1, 2,
                1, 2, 0, 2, 0, 0, 1], dtype=int)
cb = CurrentBoard(board, count_move=39, index=41, player_at_this_level=1)
stn = SearchTreeNode(Current_Game=cb)
stn.game_at_this_node.display_game_board()

stn.min_max_alpha_beta(is_max=False)
# Expected:
# 'node_terminal': False,
# 'node_value': -9223372036854775807

vars(stn)

| 1| 2| 3| 4| 5| 6| 7|
| 1| 2|  | 2|  |  | 1|
----------------------
| 2| 1| 2| 1| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 2| 2| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 1|
----------------------
| 1| 2| 1| 2| 1| 1| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 2|
----------------------


{'children': [<__main__.SearchTreeNode at 0x22ad9b796d8>],
 'ply_depth': 0,
 'max_depth': 4,
 'node_terminal': False,
 'node_trap': False,
 'node_value': -9223372036854775807,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9bdfb38>,
 'comp_first': True}

In [78]:
# Test the min layer
# Expected:
# the node_value of a parent node and node_value of one of his child should have equal value
# and the node_value of that child should be the min node_value of all children
# and the node_terminal of that child should be True

# 'node_terminal': False,
# 'node_value': -9223372036854775807, (may be different if the eval function is changed) - the same value as stn.node_value
print('PARENT: node_terminal: {}; node_value: {}'.format(stn.node_terminal, stn.node_value))

# 'node_terminal': True,
# 'node_value': -9223372036854775807, (may be different if the eval function is changed) - the same value as stn.node_value
for child in stn.children:
    print('CHILD: node_terminal: {}; node_value: {}'.format(child.node_terminal, child.node_value))

PARENT: node_terminal: False; node_value: -9223372036854775807
CHILD: node_terminal: True; node_value: -9223372036854775807


### Test 3b) get_next_move()

In [79]:
# Expected: the SearchTreeNode
# 'node_terminal': True,
# 'node_value': -9223372036854775807,
vars(stn.get_next_move())

{'children': [],
 'ply_depth': 1,
 'max_depth': 4,
 'node_terminal': True,
 'node_trap': False,
 'node_value': -9223372036854775807,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9bdfd68>,
 'comp_first': True}

In [80]:
# Expected: True
stn.node_value == stn.get_next_move().node_value 

True

### Test 4) draw situation
- index is from 0 to 41, count_move is from 0 to 42 including 42.
- player_1 makes first move, meaning that player_1 is on odd count_move, player_2 is on even count_move
### Test 4a) min_max_alpha_beta()
- the root node is a max layer

In [81]:
board = np.array([2, 2, 2, 1, 1, 2, 2,
                    1, 2, 1, 2, 1, 1, 1,
                    2, 2, 2, 1, 1, 2, 1,
                    2, 1, 2, 1, 2, 2, 1,
                    2, 1, 2, 1, 1, 1, 2,
                    1, 2, 1, 2, 0, 1, 2], dtype=int)

cb = CurrentBoard(setup_of_board=board, count_move=41, index=40 , player_at_this_level=1)
stn = SearchTreeNode(Current_Game=cb)
stn.game_at_this_node.display_game_board()

stn.min_max_alpha_beta(is_max=False)
# Expected:
# 'node_terminal': False,
# 'node_value': 0

vars(stn)

| 1| 2| 3| 4| 5| 6| 7|
| 1| 2| 1| 2|  | 1| 2|
----------------------
| 2| 1| 2| 1| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 2| 2| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 1|
----------------------
| 1| 2| 1| 2| 1| 1| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 2|
----------------------


{'children': [<__main__.SearchTreeNode at 0x22ad9b793c8>],
 'ply_depth': 0,
 'max_depth': 4,
 'node_terminal': False,
 'node_trap': False,
 'node_value': 0,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b79978>,
 'comp_first': True}

In [82]:
# Test the max layer
# Expected:
# the node_value of a parent node and node_value of one of his child should have equal value
# and the node_value of that child should be the max node_value of all children
# and the node_terminal of that child should be True

# 'node_terminal': False,
# 'node_value': 0, (may be different if the eval function is changed) - the same value as stn.node_value
print('PARENT: node_terminal: {}; node_value: {}'.format(stn.node_terminal, stn.node_value))

# 'node_terminal': True,
# 'node_value': 0, (may be different if the eval function is changed) - the same value as stn.node_value
for child in stn.children:
    print('CHILD: node_terminal: {}; node_value: {}'.format(child.node_terminal, child.node_value))

PARENT: node_terminal: False; node_value: 0
CHILD: node_terminal: True; node_value: 0


### Test 4b) get_next_move()

In [83]:
# Expected: the SearchTreeNode
# 'node_terminal': True,
# 'node_value': 0,
vars(stn.get_next_move())

{'children': [],
 'ply_depth': 1,
 'max_depth': 4,
 'node_terminal': True,
 'node_trap': False,
 'node_value': 0,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9b79518>,
 'comp_first': True}

In [84]:
# Expected: True
stn.node_value == stn.get_next_move().node_value 

True

## Test all_possible_indices()

### Test 1) default SearchNodeTree

In [85]:
stn = SearchTreeNode(max_depth=2)
# Expected: [0, 1, 2, 3, 4, 5, 6]
all_possible_indices = stn.all_possible_indices()
all_possible_indices

[0, 1, 2, 3, 4, 5, 6]

### Test 2)

In [86]:
board=np.array([2, 2, 2, 1, 1, 2, 2,
                1, 2, 1, 2, 1, 1, 1,
                2, 2, 2, 1, 1, 2, 1,
                2, 1, 2, 1, 2, 2, 1,
                2, 1, 2, 1, 1, 1, 2,
                1, 2, 0, 2, 0, 0, 2], dtype=int)
cb = CurrentBoard(board, count_move=39, index=41, player_at_this_level=1)
stn = SearchTreeNode(Current_Game=cb)

# Expected: [None, None, 37, None, 39, 40, None]
all_possible_indices = stn.all_possible_indices()
all_possible_indices

[None, None, 37, None, 39, 40, None]

## Test isCompMove()

### Test 1) default SearchTreeNode

In [87]:
stn = SearchTreeNode(max_depth=2, comp_first=True)
vars(stn)

{'children': [<__main__.SearchTreeNode at 0x22ad9b79ef0>,
 'ply_depth': 0,
 'max_depth': 2,
 'node_terminal': False,
 'node_trap': False,
 'node_value': None,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ad9be92e8>,
 'comp_first': True}

In [88]:
# Expected: False
stn.isCompMove()

False

In [89]:
# Expected True
stn.children[0].isCompMove()

True

In [90]:
# Expected False
stn.children[0].children[0].isCompMove()

False

## Test dig_up()

In [91]:
# Expected: True
# the size of SearchTreeNode after dig_up is equal to the size of SearchTreeNode with double max_depth
stn_1 = SearchTreeNode(max_depth=3, comp_first=True)
stn_1.dig_up()
tree_size_1 = stn_1.tree_size()

stn_2 = SearchTreeNode(max_depth=6, comp_first=True)
tree_size_2 = stn_2.tree_size()
tree_size_1 == tree_size_2

True

In [92]:
board = np.array([2, 2, 2, 1, 1, 2, 2,
                    1, 2, 1, 2, 1, 1, 1,
                    2, 2, 2, 1, 1, 2, 1,
                    2, 1, 2, 1, 2, 2, 1,
                    2, 1, 2, 1, 1, 1, 2,
                    1, 2, 1, 2, 1, 1, 2], dtype=int)

cb = CurrentBoard(setup_of_board=board, count_move=42, index=40 , player_at_this_level=2)
stn = SearchTreeNode(Current_Game=cb)
stn.min_max_alpha_beta()
tree_size_1 = stn.tree_size()
stn.dig_up()
tree_size_2 = stn.tree_size()

tree_size_1 == tree_size_2

True

In [93]:
board = np.array([2, 2, 2, 1, 1, 2, 2,
                    1, 2, 1, 2, 1, 1, 1,
                    2, 2, 2, 1, 1, 2, 1,
                    2, 1, 2, 1, 2, 2, 1,
                    2, 1, 2, 1, 1, 1, 2,
                    1, 2, 1, 2, 0, 1, 2], dtype=int)

cb = CurrentBoard(setup_of_board=board, count_move=41, index=40 , player_at_this_level=1)
stn = SearchTreeNode(Current_Game=cb)
stn.min_max_alpha_beta()
tree_size_1 = stn.tree_size()
stn.dig_up()
tree_size_2 = stn.tree_size()

# Expected: True
tree_size_1 == tree_size_2

True

In [94]:
board=np.array([2, 2, 2, 1, 1, 2, 2,
                1, 2, 1, 2, 1, 1, 1,
                2, 2, 2, 1, 1, 2, 1,
                2, 1, 2, 1, 2, 2, 1,
                2, 1, 2, 1, 1, 1, 2,
                1, 2, 0, 2, 0, 0, 1], dtype=int)
cb = CurrentBoard(board, count_move=39, index=41, player_at_this_level=1)
stn = SearchTreeNode(Current_Game=cb, max_depth=7)
stn.min_max_alpha_beta()
tree_size_1 = stn.tree_size()
#stn.dig_up()
tree_size_2 = stn.tree_size()

tree_size_1 == tree_size_2
print(stn.children)
stn.children[1].game_at_this_node.display_game_board()
print('CurrentBoard at this node properties:')
vars(stn)

[<__main__.SearchTreeNode object at 0x0000022AE0222940>, <__main__.SearchTreeNode object at 0x0000022AE0222978>, <__main__.SearchTreeNode object at 0x0000022AE0222A20>]
| 1| 2| 3| 4| 5| 6| 7|
| 1| 2|  | 2| 2|  | 1|
----------------------
| 2| 1| 2| 1| 1| 1| 2|
----------------------
| 2| 1| 2| 1| 2| 2| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 1|
----------------------
| 1| 2| 1| 2| 1| 1| 1|
----------------------
| 2| 2| 2| 1| 1| 2| 2|
----------------------
CurrentBoard at this node properties:


{'children': [<__main__.SearchTreeNode at 0x22ae0222940>,
 'ply_depth': 0,
 'max_depth': 7,
 'node_terminal': False,
 'node_trap': False,
 'node_value': -9223372036854775807,
 'game_at_this_node': <__main__.CurrentBoard at 0x22ae0222780>,
 'comp_first': True}

# TEST PERFORMACNE generate_children() -> min_max_alpha_beta() -> dig_up() -> min_max_alpha_beta()


### Test 1) Default SearchTreeNode 

Expected:
- the first move in the middle (the only move that guarantees winning)

##  # TODO: Test max_depth: odd max_depth did not give the best first move, the even max_depth gives best first move

In [95]:
import time

In [96]:
start_time = time.time()

stn = SearchTreeNode(max_depth=3)
stn.min_max_alpha_beta(is_max=stn.isMax())

end_time = time.time()
print(end_time - start_time)

0.017990589141845703


In [97]:
stn.get_next_move().game_at_this_node.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------


In [98]:
start_time = time.time()

stn = SearchTreeNode(max_depth=5)
stn.min_max_alpha_beta(is_max=stn.isMax())

end_time = time.time()
print(end_time - start_time)

0.8414797782897949


In [99]:
stn.get_next_move().game_at_this_node.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------


In [100]:
start_time = time.time()

stn = SearchTreeNode(max_depth=7)
stn.min_max_alpha_beta(is_max=stn.isMax())

end_time = time.time()
print(end_time - start_time)

35.67170691490173


In [101]:
stn.get_next_move().game_at_this_node.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------


In [102]:
start_time = time.time()

stn = SearchTreeNode(max_depth=6)
stn.min_max_alpha_beta(is_max=stn.isMax())

end_time = time.time()
print(end_time - start_time)

4.806014537811279


In [103]:
stn.get_next_move().game_at_this_node.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------


In [104]:
# Reslts:
# - pure stn with max_depth=5: 2 sek
# - stn max_depth=3, dig_up(2): 0.85 sek

start_time = time.time()

stn = SearchTreeNode(max_depth=3)
stn.min_max_alpha_beta(is_max=stn.isMax())
stn.dig_up(2)
stn.min_max_alpha_beta(is_max=stn.isMax())

end_time = time.time()
print(end_time - start_time)


0.2338578701019287


In [105]:
stn.get_next_move().game_at_this_node.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------


In [106]:
# Reslts:
# - pure stn with max_depth=7: 90 sek
# - stn max_depth=5, dig_up(2): 7 sek

# Much better performance but wrong result. The first move is not in the middle

start_time = time.time()

stn = SearchTreeNode(max_depth=5)
stn.min_max_alpha_beta(is_max=stn.isMax())
stn.dig_up(2)
stn.min_max_alpha_beta(is_max=stn.isMax())

end_time = time.time()
print(end_time - start_time)

2.1656582355499268


In [107]:
stn.get_next_move().game_at_this_node.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  | 1|  |  |  |  |
----------------------


In [108]:
# Reslts:
# - pure stn with max_depth=7: 90 sek
# - stn max_depth=3, dig_up(2), dig_up(2): 5 sek

# Fast but it is loosing the best move. The first move is not in the middle

start_time = time.time()

stn = SearchTreeNode(max_depth=3)
stn.min_max_alpha_beta(is_max=stn.isMax())
stn.dig_up(2)
stn.min_max_alpha_beta(is_max=stn.isMax())
stn.dig_up(2)
stn.min_max_alpha_beta(is_max=stn.isMax())

end_time = time.time()
print(end_time - start_time)

1.608001947402954


In [109]:
stn.get_next_move().game_at_this_node.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  | 1|  |  |  |  |
----------------------


In [110]:
# Expected- first move in the middle of the bottom row
# Result:
# Slow and it is still loosing the best move. 

start_time = time.time()

stn = SearchTreeNode(max_depth=5)
stn.min_max_alpha_beta(is_max=stn.isMax())
stn.dig_up(3)
stn.min_max_alpha_beta(is_max=stn.isMax())

end_time = time.time()
print(end_time - start_time)

10.534822225570679


In [111]:
stn.get_next_move().game_at_this_node.display_game_board()

| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  | 1|  |  |  |  |
----------------------


# TEST RUNNER CLASS

In [112]:
runner = Runner()
vars(runner)

{'user_name': 'Adam',
 'player_1': 1,
 'player_2': 2,
 'comp_first': True,
 'game_search_tree_node': None,
 'current_board': None,
 'result': None,
 'user_char': None,
 'comp_char': None,
 'max_depth': 3}

## Test set_up_game() method

In [113]:
# runner.set_up_game()

In [114]:
# vars(runner)

## Test first_move()

In [115]:
# runner.first_move()

In [116]:
# Expected: game_search_tree_node': <__main__.SearchTreeNode at .... >
# vars(runner)

In [117]:
# vars(runner.game_search_tree_node)

In [118]:
# vars(runner.game_search_tree_node.children[0].game_at_this_node)

In [119]:
# vars(runner.game_search_tree_node.game_at_this_node)

In [120]:
# runner.game_search_tree_node.game_at_this_node.display_game_board()

In [121]:
# runner.game_search_tree_node.next_move.game_at_this_node.display_game_board()

## Test validate_input() method

In [122]:
# runner.validate_input()

## Test loop_game()

In [123]:
# runner.loop_game()

In [124]:
# vars(runner)

### Test run()

In [125]:
# stop -> delete to run game

In [126]:
runner = Runner(max_depth=3)
runner.run()

Hello, insert your nameAdam
Would you like to start Y/Nn
In a loop_game method. Computer Move - count_move: 1; outcome is: 7; comp_char: 1, 
Board after computer move is:
| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  | 1|  |  |  |
----------------------
Adam, it is your turn. Choose column >> 
3
User Move - count_move: 2; outcome is: 5; comp_char: 1, player_at_this_level 2
Board after User move is:
| 1| 2| 3| 4| 5| 6| 7|
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  |  |  |  |  |  |
----------------------
|  |  | 2| 1|  |  |  |
----------------------
In a loop_game method. Computer Move - count_move: 3; outcome is: 9; comp_char: 1, 